In [4]:
import pandas as pd
import torch
# verificam cuda
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

True
0
NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [5]:
data = pd.read_csv('data/ml_insurance_challenge.csv')
labels = pd.read_csv('data/insurance_taxonomy - insurance_taxonomy.csv')
print(data.shape)
print(labels.shape)

(9494, 5)
(220, 1)


In [1]:
# intfloat/multilingual-e5-base
from transformers import pipeline
model = "facebook/bart-large-mnli"
classifier = pipeline("zero-shot-classification", model=model, device="cuda:0")

config.json: 0.00B [00:00, ?B/s]

C:\Users\david\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\david\.cache\huggingface\hub\models--cross-encoder--nli-deberta-v3-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Device set to use cuda:0


In [2]:
import re
from html import unescape

def soft_clean(s: str) -> str:
    if not s:
        return ""
    s = unescape(s)  # decode HTML entities
    s = re.sub(r'https?://\S+', '<URL>', s)
    s = re.sub(r'\S+@\S+', '<EMAIL>', s)
    # înlocuiește doar multiplele spații, fără a atinge newline-urile
    s = re.sub(r'[ \t]+', ' ', s)
    # normalizează liniile multiple: mai mult de 2 -> doar 2
    s = re.sub(r'\n{3,}', '\n\n', s)
    # curăță spații la capete de linii
    s = re.sub(r' *\n *', '\n', s)
    return s.strip()

In [6]:
text = ''
text += "Passage: This is a company profile\n"
for index, column in enumerate(data.iloc[0]):
    # dacă e listă de stringuri în formă text (ex: "['a','b','c']")
    if isinstance(column, str) and column.startswith('[') and column.endswith(']'):
        items = column[1:-1].split(',')
        text += "Business Tags: "
        for item in items:
            text += f"{item.strip().strip('\"').strip('\'')}, "  # eliminăm spații și ghilimele
        text = text[:-2] + '\n'  # scoate ultima virgulă, adaugă newline

    else:
        text += f"{data.columns[index].capitalize()}: {column}\n"

text = soft_clean(text)
sentence_to_classify = text
print(sentence_to_classify)
candidate_labels = labels['label'].tolist()
# adaugam "This is a insurance taxonomy describing a type of business. " la fiecare label
candidate_labels

Passage: This is a company profile
Description: Welchcivils is a civil engineering and construction company that specializes in designing and building utility network connections across the UK. They offer multi-utility solutions that combine electricity, gas, water, and fibre optic installation into a single contract. Their design engineer teams are capable of designing electricity, water and gas networks from existing network connection points to meter locations at the development, as well as project management of reinforcements and diversions. They provide custom connection solutions that take into account any existing assets, maximize the usage of every trench, and meet project deadlines. Welchcivils has considerable expertise installing gas and electricity connections in a variety of market categories, including residential, commercial, and industrial projects, as as well.
Business Tags: Construction Services, Multi-utilities, Utility Network Connections Design and Construction, Wa

['Agricultural Equipment Services',
 'Soil Nutrient Application Services',
 'Pesticide Application Services',
 'Ornamental Plant Nurseries',
 'Landscaping Services',
 'Gardening Services',
 'Tree Services - Pruning / Removal',
 'Veterinary Services',
 'Veterinary Clinics',
 'Pet Boarding Services',
 'Animal Day Care Services',
 'Pet Grooming Services',
 'Animal Training Services',
 'Veterinary Health Centers',
 'Animal Trainers',
 'Livestock Dealer Services',
 'Timber Harvesting Operations',
 'Fishing and Hunting Services',
 'Well Maintenance Services',
 'Field Welding Services',
 'Sand and Gravel Mining',
 'Residential Driveway Construction',
 'Commercial Driveway Construction',
 'Fencing Construction Services',
 'Sidewalk Construction Services',
 'Commercial Irrigation Systems',
 'Residential Drainage Systems',
 'Residential Snow Removal',
 'Commercial Snow Removal',
 'General Snow Removal Services',
 'Land Leveling Services',
 'Residential Drain Cleaning',
 'Commercial Drain Cleanin

In [7]:
classifier(sentence_to_classify, candidate_labels, multi_label=True)

{'sequence': 'Passage: This is a company profile\nDescription: Welchcivils is a civil engineering and construction company that specializes in designing and building utility network connections across the UK. They offer multi-utility solutions that combine electricity, gas, water, and fibre optic installation into a single contract. Their design engineer teams are capable of designing electricity, water and gas networks from existing network connection points to meter locations at the development, as well as project management of reinforcements and diversions. They provide custom connection solutions that take into account any existing assets, maximize the usage of every trench, and meet project deadlines. Welchcivils has considerable expertise installing gas and electricity connections in a variety of market categories, including residential, commercial, and industrial projects, as as well.\nBusiness Tags: Construction Services, Multi-utilities, Utility Network Connections Design and 

In [8]:
text = ''
text += "This is a company profile\n"
for index, column in enumerate(data.iloc[1]):
    # dacă e listă de stringuri în formă text (ex: "['a','b','c']")
    if isinstance(column, str) and column.startswith('[') and column.endswith(']'):
        items = column[1:-1].split(',')
        text += "Business Tags: "
        for item in items:
            text += f"{item.strip().strip('\"').strip('\'')}, "  # eliminăm spații și ghilimele
        text = text[:-2] + '\n'  # scoate ultima virgulă, adaugă newline

    else:
        text += f"{data.columns[index].capitalize()}: {column}\n"

text = soft_clean(text)
sentence_to_classify = text
print(sentence_to_classify)

Description: Kyoto Vegetable Specialists Uekamo, also known as Iwa-machi, is a company based in Kyoto, Japan that specializes in the sale of vegetables. They have been in business for ten years and offer a collection of vegetable recipes through their Keiō Vegetable Recipe Collection and Online Shop. The company is directly owned by Uekamoo Farm, Uekame Farm, and Lobechi Shijo-hara Farm. They offer a variety of vegetable products, including suguki-zuke and Kamoo eggplant, and also accept production cultivation according to customer requests. Iwaichi Limited Company uses their experience in production and sales to provide tailored vegetables to meet customer needs and also accepts cultivation of products according to their requirements.
Business Tags: Wholesale, Dual-task Movement Products, Cast Iron Products Manufacturer, Manufacturing Technology, Food and Beverage, Rice And Noodles, High-quality Gloss of Cast Iron, Rice Wholesaler, Miscellaneous Crop Farming, Health and Wellness Produ

In [9]:
classifier(sentence_to_classify, candidate_labels, multi_label=True)

{'sequence': 'Description: Kyoto Vegetable Specialists Uekamo, also known as Iwa-machi, is a company based in Kyoto, Japan that specializes in the sale of vegetables. They have been in business for ten years and offer a collection of vegetable recipes through their Keiō Vegetable Recipe Collection and Online Shop. The company is directly owned by Uekamoo Farm, Uekame Farm, and Lobechi Shijo-hara Farm. They offer a variety of vegetable products, including suguki-zuke and Kamoo eggplant, and also accept production cultivation according to customer requests. Iwaichi Limited Company uses their experience in production and sales to provide tailored vegetables to meet customer needs and also accepts cultivation of products according to their requirements.\nBusiness Tags: Wholesale, Dual-task Movement Products, Cast Iron Products Manufacturer, Manufacturing Technology, Food and Beverage, Rice And Noodles, High-quality Gloss of Cast Iron, Rice Wholesaler, Miscellaneous Crop Farming, Health and

In [9]:
# adaugam "This is a insurance taxonomy describing a type of business. " la fiecare label
label_texts = []
for idx in range(len(candidate_labels)):
    for index, column in enumerate(labels.iloc[idx]):
        label_text = "This is an insurance taxonomy describing a type of business. "
        label_text += column
        label_text = soft_clean(label_text)
        label_texts.append(label_text)
label_texts

['This is an insurance taxonomy describing a type of business. Agricultural Equipment Services',
 'This is an insurance taxonomy describing a type of business. Soil Nutrient Application Services',
 'This is an insurance taxonomy describing a type of business. Pesticide Application Services',
 'This is an insurance taxonomy describing a type of business. Ornamental Plant Nurseries',
 'This is an insurance taxonomy describing a type of business. Landscaping Services',
 'This is an insurance taxonomy describing a type of business. Gardening Services',
 'This is an insurance taxonomy describing a type of business. Tree Services - Pruning / Removal',
 'This is an insurance taxonomy describing a type of business. Veterinary Services',
 'This is an insurance taxonomy describing a type of business. Veterinary Clinics',
 'This is an insurance taxonomy describing a type of business. Pet Boarding Services',
 'This is an insurance taxonomy describing a type of business. Animal Day Care Services',


In [10]:
classifier(sentence_to_classify, label_texts, multi_label=True)

{'sequence': 'This is a company profile\nDescription: Kyoto Vegetable Specialists Uekamo, also known as Iwa-machi, is a company based in Kyoto, Japan that specializes in the sale of vegetables. They have been in business for ten years and offer a collection of vegetable recipes through their Keiō Vegetable Recipe Collection and Online Shop. The company is directly owned by Uekamoo Farm, Uekame Farm, and Lobechi Shijo-hara Farm. They offer a variety of vegetable products, including suguki-zuke and Kamoo eggplant, and also accept production cultivation according to customer requests. Iwaichi Limited Company uses their experience in production and sales to provide tailored vegetables to meet customer needs and also accepts cultivation of products according to their requirements.\nBusiness Tags: Wholesale, Dual-task Movement Products, Cast Iron Products Manufacturer, Manufacturing Technology, Food and Beverage, Rice And Noodles, High-quality Gloss of Cast Iron, Rice Wholesaler, Miscellaneo

In [13]:
text = ''
text += "Passage: This is a company profile\n"
for index, column in enumerate(data.iloc[4000]):
    # dacă e listă de stringuri în formă text (ex: "['a','b','c']")
    if isinstance(column, str) and column.startswith('[') and column.endswith(']'):
        items = column[1:-1].split(',')
        text += "Business Tags: "
        for item in items:
            text += f"{item.strip().strip('\"').strip('\'')}, "  # eliminăm spații și ghilimele
        text = text[:-2] + '\n'  # scoate ultima virgulă, adaugă newline

    else:
        text += f"{data.columns[index].capitalize()}: {column}\n"

text = soft_clean(text)
sentence_to_classify2 = text
print(sentence_to_classify2)

Passage: This is a company profile
Description: Bolle Safety Ultim8 is a company that specializes in producing Ultimate Glasses with Clear Lens, Black/Red/Clear, Clear, One Size products. They offer professional reviews from industry experts at Royal Designs, Inc. Their products include the Bso18Wh Flare Bottom Outside Corner Scallop Basic Lamp Shade, 10 x 18 x 13, and the Rust-Oleum Corporation Zinsser Shieldz Universal Wallcovering Primer Sealer, 1-Gallon(Package may vary).
Business Tags: Dirt Devil Power Express Lite, CLC Work Gear 1579 20 Inch
Sector: Retail
Category: Ophthalmic & Optical Instruments & Lenses
Niche: Optical Goods Retailers


In [14]:
classifier(sentence_to_classify2, label_texts, multi_label=True)

{'sequence': 'Passage: This is a company profile\nDescription: Bolle Safety Ultim8 is a company that specializes in producing Ultimate Glasses with Clear Lens, Black/Red/Clear, Clear, One Size products. They offer professional reviews from industry experts at Royal Designs, Inc. Their products include the Bso18Wh Flare Bottom Outside Corner Scallop Basic Lamp Shade, 10 x 18 x 13, and the Rust-Oleum Corporation Zinsser Shieldz Universal Wallcovering Primer Sealer, 1-Gallon(Package may vary).\nBusiness Tags: Dirt Devil Power Express Lite, CLC Work Gear 1579 20 Inch\nSector: Retail\nCategory: Ophthalmic & Optical Instruments & Lenses\nNiche: Optical Goods Retailers',
 'labels': ['This is an insurance taxonomy describing a type of business. Low-Rise Glass Installation',
  'This is an insurance taxonomy describing a type of business. Stakeholder Services',
  'This is an insurance taxonomy describing a type of business. High-Rise Glass Installation',
  'This is an insurance taxonomy describi